In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from optimus.trainer.model.encoder.biqwen import Qwen3ForMaskedLM
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

In [3]:
model_ref = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", attn_implementation = "eager")
model = Qwen3ForMaskedLM.from_pretrained("Qwen/Qwen3-0.6B")

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

text = ["Hello, how are you?", "Bonjour, comment ça va aujourd'hui, bien et vous?"]
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
print(inputs)
print(inputs.input_ids.shape, inputs.attention_mask.shape)

{'input_ids': tensor([[  9707,     11,   1246,    525,    498,     30, 151643, 151643, 151643,
         151643, 151643, 151643],
        [ 81581,     11,   3980,  37943,  11164,  74704,  87153,     11,  14370,
           1842,   9012,     30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
torch.Size([2, 12]) torch.Size([2, 12])


In [5]:
outputs_ref = model_ref(**inputs, return_dict=False)

In [6]:
print(outputs_ref[0][0][0])

tensor([5.9520, 4.9115, 3.7664,  ..., 1.4857, 1.4857, 1.4857],
       grad_fn=<SelectBackward0>)


In [7]:
def to_torch_collate_var_len_fn(batch):
    """
    Collate function for the dataloader. Prepares the batch for training with variable-length samples.
    Args:
        batch: List of tuples (input_seq, label_seq).
    Returns:
        dict[str, torch.Tensor]: A dictionary containing input_ids, labels, and cu_seq_lens.
    """
    input_seqs, label_seqs = zip(*batch)

    # Compute cumulative sequence lengths
    lengths = torch.tensor([len(seq) for seq in input_seqs], dtype=torch.int32)
    cu_seq_lens = torch.cat(
        [
            torch.zeros(1, dtype=torch.int32),
            torch.cumsum(lengths, dim=0, dtype=torch.int32),
        ]
    )

    # Concatenate inputs and labels into single tensors
    inputs = torch.cat(
        [torch.tensor(seq, dtype=torch.long) for seq in input_seqs], dim=0
    )
    labels = torch.cat(
        [torch.tensor(seq, dtype=torch.long) for seq in label_seqs], dim=0
    )

    return {
        "x": inputs,
        "labels": labels,
        "cu_seq_lens": cu_seq_lens,
        "max_seqlen": lengths.max().item(),
    }

In [8]:
batch = (
    [[9707,   11, 1246,  525,  498,   30], [9707,   11, 1246,  525,  498,   30]],
    [[ 81581,     11,   3980,  37943,  11164,  74704,  87153,     11,  14370, 1842,   9012,     30], [ 81581,     11,   3980,  37943,  11164,  74704,  87153,     11,  14370,
           1842,   9012,     30]], 
)
input_var_len = to_torch_collate_var_len_fn(batch)
# print(input_var_len)

In [9]:
input_var_len["x"].size(0)

18

In [10]:
outputs = model(
    x=input_var_len["x"],
    cu_seq_lens=input_var_len["cu_seq_lens"],
    max_seqlen=input_var_len["max_seqlen"],
)


In [11]:
print(outputs[0])

tensor([[ 6.4538,  2.9917,  0.8476,  ...,  1.2313,  1.2313,  1.2313],
        [ 1.9180,  0.0588, -2.8398,  ..., -1.7725, -1.7725, -1.7725],
        [ 3.1865,  1.4573, -2.9150,  ..., -1.4288, -1.4288, -1.4288],
        ...,
        [ 1.6484,  1.1593, -3.7017,  ..., -3.3443, -3.3443, -3.3443],
        [10.1019,  2.5858, -0.0210,  ..., -1.8892, -1.8892, -1.8892],
        [ 5.6756, -1.2023, -1.0030,  ..., -0.8785, -0.8785, -0.8785]],
       grad_fn=<MmBackward0>)


---

In [29]:
import torch
from dev.hf_model.modeling_biqwen import BiQwen3ForMaskedLM

from configuration_biqwen import BiQwen3Config

config = BiQwen3Config.from_pretrained("Qwen/Qwen3-0.6B")
config.architectures = ["BiQwen3ForMaskedLM"]
# Charger ton modèle Hugging Face (même architecture que celui entraîné)
model = BiQwen3ForMaskedLM(config)

# Charger les poids
state_dict = torch.load("/Users/nboizard/Documents/Decoder2Encoder/dev/model.pt", map_location="cpu")
model.load_state_dict(state_dict)

You are using a model of type qwen3 to instantiate a model of type biqwen3. This is not supported for all configurations of models and can yield errors.
/var/folders/3b/s9vbnkf14jj2vpl7r_kzytym0000gn/T/ipykernel_3811/1893946906.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full contro

<All keys matched successfully>

In [28]:
config

BiQwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 40960,
  "max_window_layers": 28,
  "model_type": "biqwen3",
  "num_attention_heads": 16,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.54.1",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

In [23]:
model.save_pretrained("/Users/nboizard/Documents/Decoder2Encoder/dev/model")

In [15]:
model(
    input_ids=input_var_len["x"],
    cu_seqlens=input_var_len["cu_seq_lens"],
    max_seqlen=input_var_len["max_seqlen"],
)

(tensor([[13.3115,  5.6015,  3.1471,  ..., -6.5168, -6.5168, -6.5168],
         [ 7.1337,  2.1764, -1.1518,  ..., -7.9029, -7.9029, -7.9029],
         [ 6.7046,  4.4314, -2.5052,  ..., -7.7371, -7.7371, -7.7371],
         ...,
         [ 2.6096,  3.6805, -3.3847,  ..., -5.7916, -5.7916, -5.7916],
         [11.7999,  4.7332, -2.7367,  ..., -6.4062, -6.4062, -6.4062],
         [10.3729,  3.2369,  6.1813,  ..., -8.8299, -8.8299, -8.8299]],
        grad_fn=<MmBackward0>),
 None)

In [16]:
model(**inputs)

(tensor([[[13.3115,  5.6015,  3.1471,  ..., -6.5168, -6.5168, -6.5168],
          [ 7.1337,  2.1764, -1.1518,  ..., -7.9029, -7.9029, -7.9029],
          [ 6.7046,  4.4314, -2.5052,  ..., -7.7371, -7.7371, -7.7371],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[12.9486,  6.7320,  1.0511,  ..., -7.6356, -7.6356, -7.6356],
          [ 8.0272,  2.6425,  0.8983,  ..., -8.3366, -8.3366, -8.3366],
          [ 3.9991,  0.6889, -1.6330,  ..., -7.6215, -7.6215, -7.6215],
          ...,
          [ 2.6096,  3.6805, -3.3847,  ..., -5.7916, -5.7916, -5.7916],
          [11.7999,  4.7332, -2.7367,  ..., -6.4062, -6.4062, -6.4062],
          [10.3729,  3.2369,  6.1813,  ..., -8.8299, -8.8299, -8.8299]]],
        grad_fn=<UnsafeViewBackward0>),
 None)

In [17]:
model.save_pretrained("/Users/nboizard/Documents/Decoder2Encoder/dev/model")

In [18]:
from transformers.models.qwen3.configuration_qwen3 import Qwen3Config

In [19]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("/Users/nboizard/Documents/Decoder2Encoder/dev/hf_model", trust_remote_code=True)

In [21]:
model(**inputs)

(tensor([[[13.3115,  5.6015,  3.1471,  ..., -6.5168, -6.5168, -6.5168],
          [ 7.1337,  2.1764, -1.1518,  ..., -7.9029, -7.9029, -7.9029],
          [ 6.7046,  4.4314, -2.5052,  ..., -7.7371, -7.7371, -7.7371],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[12.9486,  6.7320,  1.0511,  ..., -7.6356, -7.6356, -7.6356],
          [ 8.0272,  2.6425,  0.8983,  ..., -8.3366, -8.3366, -8.3366],
          [ 3.9991,  0.6889, -1.6330,  ..., -7.6215, -7.6215, -7.6215],
          ...,
          [ 2.6096,  3.6805, -3.3847,  ..., -5.7916, -5.7916, -5.7916],
          [11.7999,  4.7332, -2.7367,  ..., -6.4062, -6.4062, -6.4062],
          [10.3729,  3.2369,  6.1813,  ..., -8.8299, -8.8299, -8.8299]]],
        grad_fn=<UnsafeViewBackward0>),
 None)

---

In [ ]:
%load_ext autoreload
%autoreload 2

In [36]:
import torch
from dev.hf_model.modeling_biqwen import BiQwen3ForMaskedLM

from configuration_biqwen import BiQwen3Config

config = BiQwen3Config.from_pretrained("/Users/nboizard/Documents/Decoder2Encoder/optimus/tools/huggingface/biqwen/600m.json")
# config.architectures = ["BiQwen3ForMaskedLM"]
# # Charger ton modèle Hugging Face (même architecture que celui entraîné)
# model = BiQwen3ForMaskedLM(config)

# # Charger les poids
# state_dict = torch.load("/Users/nboizard/Documents/Decoder2Encoder/dev/model.pt", map_location="cpu")
# model.load_state_dict(state_dict)

In [31]:
model

BiQwen3ForMaskedLM(
  (model): BiQwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3EncoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm):

In [32]:
model.config

BiQwen3Config {
  "architectures": [
    "BiQwen3ForMaskedLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 40960,
  "max_window_layers": 28,
  "model_type": "biqwen3",
  "num_attention_heads": 16,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.54.1",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

In [35]:
model.save_pretrained("/Users/nboizard/Documents/Decoder2Encoder/dev/model")

In [33]:
model.push_to_hub("test", organization="Dec2Enc", private=True)

/Users/nboizard/miniconda3/envs/Decoder2Encoder/lib/python3.11/site-packages/transformers/utils/hub.py:748: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(
Processing Files (0 / 0)                : |          |  0.00B /  0.00B            huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing Files (0 / 1)                :   0%|          |  551kB / 2.38GB,  689kB/s  
Processing Files (0 / 1)                :   0%|          | 1.65MB / 2.38GB, 1.65MB/s  
Processing Files (0 / 1)                :   0%|          | 4.96MB / 2.38GB, 4.13MB/s  
Proces

CommitInfo(commit_url='https://huggingface.co/Dec2Enc/test/commit/5c2b47244e46a588cb0890914f0dc900cdf3bd43', commit_message='Upload BiQwen3ForMaskedLM', commit_description='', oid='5c2b47244e46a588cb0890914f0dc900cdf3bd43', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dec2Enc/test', endpoint='https://huggingface.co', repo_type='model', repo_id='Dec2Enc/test'), pr_revision=None, pr_num=None)

In [34]:
# Load model directly
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("Dec2Enc/test", torch_dtype="auto"),

ValueError: The checkpoint you are trying to load has model type `biqwen3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

---

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
model = AutoModelForMaskedLM.from_pretrained("Dec2Enc/BiQwen3-0.6B-lr-xp", trust_remote_code=True, revision="0.30_lr5.00000000e-04")
sentenceModel = SentenceTransformer("Dec2Enc/BiQwen3-0.6B-lr-xp", trust_remote_code=True, revision="0.30_lr5.00000000e-04")
tokenizer = AutoTokenizer.from_pretrained("Dec2Enc/BiQwen3-0.6B-lr-xp", trust_remote_code=True, revision="0.30_lr5.00000000e-04")

text = ["Hello, how are you?", "Bonjour, comment ça va aujourd'hui, bien et vous?"]
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
print(inputs)
print(inputs.input_ids.shape, inputs.attention_mask.shape)

{'input_ids': tensor([[  9707,     11,   1246,    525,    498,     30, 151643, 151643, 151643,
         151643, 151643, 151643],
        [ 81581,     11,   3980,  37943,  11164,  74704,  87153,     11,  14370,
           1842,   9012,     30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
torch.Size([2, 12]) torch.Size([2, 12])


In [12]:
model

BiQwen3ForMaskedLM(
  (model): BiQwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3EncoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm):

In [13]:
model(**inputs, output_attentions=True, output_hidden_states=False)

MaskedLMOutput(loss=None, logits=tensor([[[ 9.1542,  2.6016,  1.4778,  ..., -5.5217, -5.5217, -5.5217],
         [ 7.0803,  1.5699,  0.6892,  ..., -6.0266, -6.0266, -6.0266],
         [ 5.9616,  3.0166, -0.2726,  ..., -5.4921, -5.4921, -5.4921],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 9.3462,  4.3756, -1.2847,  ..., -5.2817, -5.2817, -5.2817],
         [ 5.1010, -0.0444, -0.9785,  ..., -6.3011, -6.3011, -6.3011],
         [ 5.8530,  1.6716, -0.5194,  ..., -5.6626, -5.6626, -5.6626],
         ...,
         [ 3.3797,  3.0332, -2.9245,  ..., -5.0121, -5.0121, -5.0121],
         [11.2908,  3.7537, -3.1134,  ..., -4.5619, -4.5619, -4.5619],
         [ 9.7178,  1.0106,  3.8347,  ..., -6.1085, -6.1085, -6.1085]]],
       grad_fn=<UnsafeViewBackward0>), hidden_states=None, attentions=(tensor([[[[1

---

In [6]:
from transformers import AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("/Users/nboizard/Documents/Decoder2Encoder/dev/model")

In [8]:
len(tokenizer)

151669

In [9]:
tokenizer.mask_token

'<|mask|>'

In [10]:
tokenizer.tokenize("Je fais un test pour <|mask|> si le mask model fonctionne bien.")

['Je',
 'Ġfais',
 'Ġun',
 'Ġtest',
 'Ġpour',
 ' <|mask|>',
 'Ġsi',
 'Ġle',
 'Ġmask',
 'Ġmodel',
 'Ġfonction',
 'ne',
 'Ġbien',
 '.']

In [71]:
tokenizer.vocab_size

151643

In [70]:
tokenizer.add_tokens(["<mask>"], special_tokens=True)

1

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator

In [17]:
sentenceModel = SentenceTransformer("Dec2Enc/BiQwen3-0.6B-lr-xp", trust_remote_code=True, revision="0.30_lr1.07721735e-04")

No sentence-transformers model found with name Dec2Enc/BiQwen3-0.6B-lr-xp. Creating a new one with mean pooling.


In [18]:
sentenceModel

SentenceTransformer(
  (0): Transformer({'max_seq_length': 40960, 'do_lower_case': False, 'architecture': 'BiQwen3Model'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [4]:
import os
import random

from datasets import Dataset, DatasetDict, concatenate_datasets
from datasets import get_dataset_config_names as _get_dataset_config_names
from datasets import get_dataset_split_names as _get_dataset_split_names
from datasets import load_dataset as load_dataset
import pandas as pd
import random
from tqdm import tqdm

In [5]:
def split_retrieval_dataset(dataset, train_size=0.95, seed=42, shard_size=10_000):
    random.seed(seed)
    anchors = sorted(list(set(dataset["anchor"])))
    anchors_train = random.sample(anchors, round(train_size * len(anchors)))
    dataset_train, dataset_test = [], []
    for i in tqdm(range(0, len(dataset), shard_size)):
        shard = pd.DataFrame(dataset[i:i+shard_size])
        is_train = shard["anchor"].isin(anchors_train)
        dataset_train.append(Dataset.from_pandas(shard.loc[is_train].reset_index(drop=True)))
        dataset_test.append(Dataset.from_pandas(shard.loc[~is_train].reset_index(drop=True)))
    dataset_train = concatenate_datasets(dataset_train)
    dataset_test = concatenate_datasets(dataset_test)
    return dataset_train, dataset_test

In [6]:
dataset = load_dataset("bclavie/msmarco-10m-triplets")
dataset = dataset.rename_column("query", "anchor")
dataset = dataset.map(
    lambda x: {
        "anchor": f"Query: {x['anchor']}", 
        "positive": f"Document: {x['positive']}", 
        "negative": f"Document: {x['negative']}",
    }
)
dataset["train"], dataset_val_test = split_retrieval_dataset(dataset["train"], train_size=0.9, seed=42)
dataset["validation"], dataset["test"] = split_retrieval_dataset(dataset_val_test, train_size=0.5, seed=42)
dataset["train"] = DatasetDict({"en": dataset["train"]})

100%|██████████| 101/101 [00:04<00:00, 24.89it/s]


In [7]:
dataset = dataset["train"]

In [10]:
dataset_dict = {}
for k in dataset:
    dataset_dict[f"msmarco-{k}"] = dataset[k].select(range(1_000_000))
dataset_dict = DatasetDict(dataset_dict)
dataset = DatasetDict({"train": dataset_dict, "validation": None, "test": None})

In [12]:
train_dataset=dataset["train"]

In [19]:
loss = MultipleNegativesRankingLoss(sentenceModel)

args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="output/",
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=2,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if GPU can't handle FP16
    bf16=False,  # Set to True if GPU supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    logging_steps=1,
)

# dev_evaluator = TripletEvaluator(
#     anchors=eval_dataset["anchor"],
#     positives=eval_dataset["positive"],
#     negatives=eval_dataset["negative"],
#     name="all-nli-dev",
#     batch_size=2,
# )
# dev_evaluator(sentenceModel)

In [ ]:
trainer = SentenceTransformerTrainer(
    model=sentenceModel,
    args=args,
    train_dataset=train_dataset,
    loss=loss,
    # evaluator=dev_evaluator,
)
trainer.train()

Step,Training Loss
1,1.886500


In [1]:
from transformers import AutoModelForSequenceClassification

/Users/nboizard/miniconda3/envs/Decoder2Encoder/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
AutoModelForSequenceClassification.from_pretrained("Dec2Enc/BiQwen3-0.6B-lr-xp", trust_remote_code=True, revision="0.30_lr1.07721735e-04")

Some weights of BiQwen3ForSequenceClassification were not initialized from the model checkpoint at Dec2Enc/BiQwen3-0.6B-lr-xp and are newly initialized: ['classifier.bias', 'classifier.weight', 'dense.bias', 'dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BiQwen3ForSequenceClassification(
  (model): BiQwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3EncoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attenti